In [1]:
# load movie review dataset(with NER Metadata) - size = 7816
import pandas as pd
matched_reviews = pd.read_csv("clean_dataset.csv")

# print(matched_reviews.tail())
# print(matched_reviews.shape)

In [74]:
# Baseline Classification Models without NER Metadata

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Prepare data for modeling
df = matched_reviews.copy()

# Ensure text is string
df["text"] = df["text"].astype(str)

X = df["text"]
y = df["label"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(
    max_features=20_000,
    ngram_range=(1, 2),
    stop_words="english"
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)



In [75]:
# Logistic Regression Model (Without NER Metadata)
log_reg = LogisticRegression(max_iter=500)
log_reg.fit(X_train_vec, y_train)

# Evaluation
pred_lr = log_reg.predict(X_test_vec)

print("==== Logistic Regression (Without NER Metadata)====")
print("Accuracy:", accuracy_score(y_test, pred_lr))
print("F1 Score:", f1_score(y_test, pred_lr))
print(classification_report(y_test, pred_lr))

==== Logistic Regression (Without NER Metadata)====
Accuracy: 0.7595907928388747
F1 Score: 0.8493589743589743
              precision    recall  f1-score   support

           0       0.74      0.28      0.41       459
           1       0.76      0.96      0.85      1105

    accuracy                           0.76      1564
   macro avg       0.75      0.62      0.63      1564
weighted avg       0.76      0.76      0.72      1564



In [76]:
# Linear SVM Model Without NER Metadata
svm_clf = LinearSVC()
svm_clf.fit(X_train_vec, y_train)

# Evaluation
pred_svm = svm_clf.predict(X_test_vec)

print("==== Linear SVM (Without NER Metadata) ====")
print("Accuracy:", accuracy_score(y_test, pred_svm))
print("F1 Score:", f1_score(y_test, pred_svm))
print(classification_report(y_test, pred_svm))


==== Linear SVM (Without NER Metadata) ====
Accuracy: 0.7749360613810742
F1 Score: 0.8503401360544217
              precision    recall  f1-score   support

           0       0.67      0.46      0.55       459
           1       0.80      0.90      0.85      1105

    accuracy                           0.77      1564
   macro avg       0.74      0.68      0.70      1564
weighted avg       0.76      0.77      0.76      1564



In [ ]:
# Baseline Classification Models with NER Metadata
from textblob import TextBlob

# A function to compute entity features eg. count the number of time actors/directors were mentioned, entity sentiment, etc
def compute_entity_features(row):

    # actors/directors are lists, not strings
    actors = row.get("actors", [])
    directors = row.get("directors", [])

    num_actors = len(actors)
    num_directors = len(directors)

    # review text to lowercase
    text = row["text"].lower()

    # Count actor mentions
    actor_mentions = 0
    for a in actors:
        actor_mentions += text.count(a.lower())

    # Count director mentions
    director_mentions = 0
    for d in directors:
        director_mentions += text.count(d.lower())

    # Sentiment toward entity names
    entity_tokens = actors + directors
    entity_sentiment = 0

    if entity_tokens:
        combined = " ".join(entity_tokens)
        try:
            entity_sentiment = TextBlob(combined).sentiment.polarity
        except:
            entity_sentiment = 0

    return pd.Series({
        # "num_titles": num_titles,
        "num_actors": num_actors,
        "num_directors": num_directors,
        "actor_mentions": actor_mentions,
        "director_mentions": director_mentions,
        "entity_sentiment": entity_sentiment
    })


# Compute entity features
entity_features = matched_reviews.apply(compute_entity_features, axis=1)
full_df = pd.concat([matched_reviews, entity_features], axis=1)
print(full_df.tail())

In [94]:
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler

# Split BEFORE vectorization
X_text_raw = full_df["text"]
X_entity = full_df[[
    "num_actors", "num_directors",
    "actor_mentions", "director_mentions",
    "entity_sentiment" #  "num_titles",
]].fillna(0)
y = full_df["label"]

# Train-test split on raw data
X_text_train, X_text_test, X_entity_train, X_entity_test, y_train, y_test = train_test_split(
    X_text_raw, X_entity, y, test_size=0.2, random_state=42, stratify=y
)

print(X_entity_test.head())
print(X_text_test.shape)
# x_with_entity_text = [X_text_test, X_entity_test]
# x_with_entity_text.columns

# Now fit TF-IDF only on training text
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_text = tfidf.fit_transform(X_text_train)  # Learn from train only
X_test_text = tfidf.transform(X_text_test)        # Apply to test

# Scale entity features (fit on train, transform both)
scaler = StandardScaler(with_mean=False)  # Sparse-compatible
X_entity_train_scaled = scaler.fit_transform(X_entity_train.values)
X_entity_test_scaled = scaler.transform(X_entity_test.values)

# Combine features
X_train = hstack([X_train_text, X_entity_train_scaled])
X_test = hstack([X_test_text, X_entity_test_scaled])

      num_actors  num_directors  actor_mentions  director_mentions  \
1414        33.0            2.0           171.0                0.0   
5940        20.0            2.0           216.0                0.0   
6152        14.0            2.0            68.0                0.0   
57           2.0            2.0             0.0                0.0   
4423         2.0            2.0             0.0                0.0   

      entity_sentiment  
1414               0.0  
5940               0.0  
6152               0.0  
57                 0.0  
4423               0.0  
(1564,)


In [79]:
# Logistic Regression with NER features
log_clf = LogisticRegression(max_iter=500)
log_clf.fit(X_train, y_train)

pred_log = log_clf.predict(X_test)

print("==== Entity-Aware Logistic Regression ====")
print("Accuracy:", accuracy_score(y_test, pred_log))
print("F1 Score:", f1_score(y_test, pred_log))
print(classification_report(y_test, pred_log))

==== Entity-Aware Logistic Regression ====
Accuracy: 0.7774936061381074
F1 Score: 0.8557213930348259
              precision    recall  f1-score   support

           0       0.72      0.40      0.51       459
           1       0.79      0.93      0.86      1105

    accuracy                           0.78      1564
   macro avg       0.75      0.67      0.68      1564
weighted avg       0.77      0.78      0.76      1564



In [80]:
# SVM with NER features
svm_clf = LinearSVC()
svm_clf.fit(X_train, y_train)

pred_svm = svm_clf.predict(X_test)

print("==== Entity-Aware SVM ====")
print("Accuracy:", accuracy_score(y_test, pred_svm))
print("F1 Score:", f1_score(y_test, pred_svm))
print(classification_report(y_test, pred_svm))

==== Entity-Aware SVM ====
Accuracy: 0.7934782608695652
F1 Score: 0.8592592592592593
              precision    recall  f1-score   support

           0       0.68      0.56      0.61       459
           1       0.83      0.89      0.86      1105

    accuracy                           0.79      1564
   macro avg       0.76      0.72      0.74      1564
weighted avg       0.79      0.79      0.79      1564



In [81]:
import torch
torch.backends.cudnn.benchmark = True

# -------------------------
# LOAD DATA
# -------------------------
entity_cols = ["num_actors", "num_directors",
               "actor_mentions", "director_mentions", "entity_sentiment"]

df = full_df[["text", "label"] + entity_cols].dropna()
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])

from datasets import Dataset
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

# -------------------------
# TOKENIZER
# -------------------------
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

MAX_LEN = 128   # MUCH FASTER (cut 256 → 128)

def tokenize_batch(batch):
    encoded = tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LEN,
        padding=False,       # dynamic padding enabled later
    )
    # add entity features
    for col in entity_cols:
        encoded[col] = batch[col]
    return encoded

# Remove original columns except what we return
cols_to_keep = ["input_ids", "attention_mask", "label"] + entity_cols

train_ds = train_ds.map(
    tokenize_batch,
    batched=True,
    remove_columns=[c for c in train_ds.column_names if c not in cols_to_keep]
)

test_ds = test_ds.map(
    tokenize_batch,
    batched=True,
    remove_columns=[c for c in test_ds.column_names if c not in cols_to_keep]
)


train_ds = train_ds.rename_column("label", "labels")
test_ds = test_ds.rename_column("label", "labels")
print(train_ds)



Map: 100%|██████████| 1564/1564 [00:00<00:00, 23167.30 examples/s]

Dataset({
    features: ['labels', 'num_actors', 'num_directors', 'actor_mentions', 'director_mentions', 'entity_sentiment', 'input_ids', 'attention_mask'],
    num_rows: 6252
})


In [82]:
# -------------------------
# NORMALIZE ENTITY FEATURES
# -------------------------
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[entity_cols] = scaler.fit_transform(df[entity_cols])

# -------------------------
# DYNAMIC PADDING - Faster GPU and less memory
# -------------------------
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# -------------------------
# MODEL
# -------------------------
from transformers import DistilBertModel
import torch.nn as nn
import torch

class DistilBertWithEntities(nn.Module):
    def __init__(self, num_labels, entity_dim=6):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(768 + entity_dim, num_labels)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        labels=None,
        # num_titles=None,
        num_actors=None,
        num_directors=None,
        actor_mentions=None,
        director_mentions=None,
        entity_sentiment=None,
    ):
        # DistilBERT forward
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # CLS token

        # Stack entity features
        entity_tensors = [
            # num_titles.unsqueeze(1).float(),
            num_actors.unsqueeze(1).float(),
            num_directors.unsqueeze(1).float(),
            actor_mentions.unsqueeze(1).float(),
            director_mentions.unsqueeze(1).float(),
            entity_sentiment.unsqueeze(1).float(),
        ]
        entity_tensor = torch.cat(entity_tensors, dim=1)

        # Combine text + entity features
        combined = torch.cat((pooled_output, entity_tensor), dim=1)
        combined = self.dropout(combined)
        logits = self.fc(combined)

        # Loss with optional class weights
        loss = None
        if labels is not None:
            class_counts = torch.bincount(labels)
            class_weights = (1.0 / class_counts.float()).to(logits.device)
            loss_fn = nn.CrossEntropyLoss(weight=class_weights)
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits}

num_labels = df["label"].nunique()
model = DistilBertWithEntities(num_labels, entity_dim=len(entity_cols))

# -------------------------
# TRAINING ARGS (IMPROVED)
# -------------------------
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./distilbert-entity-improved",

    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,

    gradient_accumulation_steps=2,   # effective batch = 32
    learning_rate=3e-5,

    num_train_epochs=2,
    # evaluation_strategy="epoch",
    save_strategy="epoch",

    fp16=True,
    optim="adamw_torch",
    dataloader_num_workers=4,
    dataloader_pin_memory=True,

    logging_steps=100,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=data_collator,
)



In [83]:
# -------------------------
# TRAIN
# -------------------------
trainer.train()

C:\Users\charl\AppData\Roaming\Python\Python314\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
100,0.666000
200,0.490900
300,0.313100


C:\Users\charl\AppData\Roaming\Python\Python314\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=392, training_loss=0.44208768922455455, metrics={'train_runtime': 811.6499, 'train_samples_per_second': 15.406, 'train_steps_per_second': 0.483, 'total_flos': 0.0, 'train_loss': 0.44208768922455455, 'epoch': 2.0})

In [84]:
# -------------------------
# Evaluate
# -------------------------
trainer.evaluate()


C:\Users\charl\AppData\Roaming\Python\Python314\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.34162789583206177,
 'eval_runtime': 49.2761,
 'eval_samples_per_second': 31.74,
 'eval_steps_per_second': 0.994,
 'epoch': 2.0}

In [85]:
# Get predictions from trainer
predictions = trainer.predict(test_ds)

# Extract logits and labels
preds = predictions.predictions
labels = predictions.label_ids

# Convert logits to predicted class indices
preds = preds.argmax(axis=-1)

acc = accuracy_score(labels, preds)
f1 = f1_score(labels, preds, average='weighted')  # or 'macro' if you prefer

print("==== Entity Aware - DistilBERT Evaluation ====")
print("Accuracy:", acc)
print("F1 Score:", f1)
print("\nClassification Report:\n")
print(classification_report(labels, preds))

C:\Users\charl\AppData\Roaming\Python\Python314\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


==== Entity Aware - DistilBERT Evaluation ====
Accuracy: 0.8759590792838875
F1 Score: 0.877663883797496

Classification Report:

              precision    recall  f1-score   support

           0       0.76      0.84      0.80       459
           1       0.93      0.89      0.91      1105

    accuracy                           0.88      1564
   macro avg       0.85      0.87      0.85      1564
weighted avg       0.88      0.88      0.88      1564



In [97]:
# Bias Evaluation - LR and SVM
entity_cols = ["num_actors", "num_directors",
               "actor_mentions", "director_mentions", "entity_sentiment"]

entity_name_map = {
    "num_actors": "Actor Mentions",
    "num_directors": "Director Mentions",
    "actor_mentions": "Specific Actor Mentions",
    "director_mentions": "Specific Director Mentions",
    "entity_sentiment": "Entity Sentiment Score"
}

X_entity_test_df = X_entity_test[entity_cols]
print(X_entity_test_df.shape)

(1564, 5)


In [67]:
# # Split into text features and entity features
# X_text = vectorizer.transform(X_text_test)   # sparse matrix
# X_entities = X_entity_test                     # pandas DataFrame
#
# # Combine for training
# from scipy.sparse import hstack
# X_train = hstack([X_text, X_entities.values])

In [140]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

def evaluate_bias(model, X, y, entity_df, entity_cols, entity_name_map, model_name="Model"):
    preds = model.predict(X)
    acc = accuracy_score(y, preds)
    f1 = f1_score(y, preds, average="weighted")
    print(f"\n==== {model_name} ====")
    print("Accuracy:", acc)
    print("F1 Score:", f1)
    print(classification_report(y, preds))

    # Subgroup metrics
    for col in entity_cols:
        subgroup_idx = entity_df[entity_df[col] > 0].index.to_list()
        print("sub_idx", len(subgroup_idx))

        if len(subgroup_idx) > 0:
            # Use .take() for sparse matrices
            subgroup_X = X.tocsr()[subgroup_idx, :]
            subgroup_preds = model.predict(subgroup_X)
            subgroup_true = y.iloc[subgroup_idx]
            print(f"{entity_name_map[col]} → Acc: {accuracy_score(subgroup_true, subgroup_preds):.3f}, "
                  f"F1: {f1_score(subgroup_true, subgroup_preds, average='weighted'):.3f}")

# def evaluate_bias(model, X, y, entity_df, entity_name_map, model_name="Model"):
#     preds = model.predict(X)
#     acc = accuracy_score(y, preds)
#     f1 = f1_score(y, preds, average="weighted")
#     print(f"\n==== {model_name} ====")
#     print("Accuracy:", acc)
#     print("F1 Score:", f1)
#     print(classification_report(y, preds))
#
#     # Subgroup metrics
#     for col in X_entities:
#         subgroup_idx = entity_df[entity_df[col] > 0].index
#         if len(subgroup_idx) > 0:
#             subgroup_preds = model.predict(X[subgroup_idx])
#             subgroup_true = y.iloc[subgroup_idx]
#             print(f"{entity_name_map[col]} → Acc: {accuracy_score(subgroup_true, subgroup_preds):.3f}, "
#                   f"F1: {f1_score(subgroup_true, subgroup_preds, average='weighted'):.3f}")

    # for col in entity_cols:
    #     subgroup = X[X[col] > 0]
    #     if len(subgroup) > 0:
    #         subgroup_preds = model.predict(subgroup)
    #         subgroup_acc = accuracy_score(y.loc[subgroup.index], subgroup_preds)
    #         subgroup_f1 = f1_score(y.loc[subgroup.index], subgroup_preds, average="weighted")
    #         print(f"{entity_name_map[col]} → Acc: {subgroup_acc:.3f}, F1: {subgroup_f1:.3f}")

In [141]:
evaluate_bias(log_clf, X_test, y_test, X_entity_test_df, entity_cols, entity_name_map, "Entity-Aware Logistic Regression - Bias Evaluation")
evaluate_bias(svm_clf, X_test, y_test, X_entity_test_df, entity_cols, entity_name_map, "Entity-Aware SVM - Bias Evaluation")


==== Entity-Aware Logistic Regression - Bias Evaluation ====
Accuracy: 0.7774936061381074
F1 Score: 0.7554237556448992
              precision    recall  f1-score   support

           0       0.72      0.40      0.51       459
           1       0.79      0.93      0.86      1105

    accuracy                           0.78      1564
   macro avg       0.75      0.67      0.68      1564
weighted avg       0.77      0.78      0.76      1564

sub_idx 1564


IndexError: index (7813) out of range